In [1]:
# import libraries
import pandas as pd 
import itertools
from itertools import product
import numpy as np
from itertools import combinations
from operator import itemgetter

A= ['carrier', 'origin', 'destination'] 
M= ['arrivaldelay', 'departurdelay', 'weatherdelay']
F= ['SUM', 'AVG', 'MIN', 'MAX', 'STD']

df = pd.DataFrame(list(product(A,M,F)), columns=['Attributes', 'Meassure', 'Function'])
df['Utility'] = np.random.uniform(0, 0.5, df.shape[0])
df.head()

,Attributes,Meassure,Function,Utility
0,carrier,arrivaldelay,SUM,0.367909
1,carrier,arrivaldelay,AVG,0.264677
2,carrier,arrivaldelay,MIN,0.249658
3,carrier,arrivaldelay,MAX,0.197416
4,carrier,arrivaldelay,STD,0.158475


In [2]:
new_df = df.sample(4)

In [3]:
new_df

,Attributes,Meassure,Function,Utility
34,destination,arrivaldelay,STD,0.202468
30,destination,arrivaldelay,SUM,0.058358
43,destination,weatherdelay,MAX,0.134349
11,carrier,weatherdelay,AVG,0.081595


In [4]:
result = df[~df.index.isin(new_df.index)]

In [5]:
def check_attribute(attr):
    time_att = ['year','quarter','month','week','day']
    loc_att = ['country','state','region','city']
    product_att = ['category','subcategory','product_name']
    
    if attr in time_att:
        return 1
    elif attr in loc_att:
        return 2
    elif attr in product_att:
        return 3
    else:
        return 0

def time_hierarchy(at1, at2):
    time_att = ['year','quarter','month','week','day']
    val = [1,2,3,4,5]
    dict_time = dict(zip(time_att,val))
    d = abs(dict_time[at1] - dict_time[at2])
    return float(d)/len(time_att)
    
def loc_hierarchy(at1, at2):
    loc_att = ['country','state','region','city']
    val = [1,2,3,4]
    dict_loc = dict(zip(loc_att,val))
    d = abs(dict_loc[at1] - dict_loc[at2])
    return float(d)/len(loc_att)

def product_hierarchy(at1, at2):
    product_att = ['category','subcategory','product_name']
    val = [1,2,3]
    dict_prod = dict(zip(product_att,val))
    d = abs(dict_prod[at1] - dict_prod[at2])
    return float(d)/len(product_att)

def diversity(a,b):
    la = list(a)
    lb = list(b)
    v1 = get_value_d(la[0],lb[0])
    v2 = get_value_d(la[1],lb[1])
    v3 = get_value_d(la[2],lb[2])
    res = float((v1+v2+v3))/3
    return res
    
def get_value_d(attr1, attr2):
    d = None 
    if check_attribute(attr1) != 0:
        if check_attribute(attr1) == check_attribute(attr2):
            if check_attribute(attr1) == 1:
                d = time_hierarchy(attr1, attr2)
            elif check_attribute(attr1) == 2:
                d = loc_hierarchy(attr1, attr2)
            elif check_attribute(attr1) == 3:
                d = product_hierarchy(attr1, attr2)
            else:
                pass
        else:
            if (attr1) == (attr2):
                d = 0.0
            else:
                d = 1.0
    else:
        if (attr1) == (attr2):
                d = 0.0
        else:
            d = 1.0
    return d

# def diversity_bruteforce(a,b):
#     c = set(a).intersection(b)
#     d = float(len(c)) / (len(a) + len(b) - len(c))
#     return 1 - d

In [6]:
def most_distant_two_views(data):
    max_distance = 0
    result = []
    for i in range(0,len(data)):
        for j in range(0,len(data)):
            d = diversity(data[i],data[j])
            if d > max_distance:
                max_distance = d
                result = [data[i],data[j]]
    return result

def dist(X, S):
    # max_distance = []
    # for i in range(0,len(X)):
    #     d = 0
    #     for j in range(0,len(S)):
    #         d += (diversity(X[i],S[j]))/len(S)
    #     max_distance.append((d, tuple([X[i],S[j]])))
    # max_distance.sort(key=lambda x: x[0], reverse=True)
    # data = max_distance[:1]
    # return data[0][1][0]
    max_distance = 0
    result = []
    for i in range(0,len(X)):
        d = 0
        for j in range(0,len(S)):
            d += diversity(set(X[i]),set(S[j]))
            if d > max_distance:
                max_distance = d
                result = X[i]
    return result

In [7]:
def util_func_d(S_list):
    new_df = pd.DataFrame(S_list)
    utility = sum(new_df[3])/len(new_df)
    return utility

def div_func_d(S_list):
    k = len(S_list)
    new_df = pd.DataFrame(S_list)
    df_now = new_df.drop([3], axis=1)
    S_list_now = df_now.values.tolist()
    new_series = pd.Series(S_list_now)
    series_set = new_series.apply(lambda row: set(row))
    new_df = series_set.apply(lambda a: series_set.apply(lambda b: diversity(a,b)))
    new_df['tot'] = new_df.sum(axis=1)
    div = (sum(new_df['tot'])/2)/(k*(k-1))
    return div

def objf_func_d(S):
    util = util_func_d(S)
    div = div_func_d(S)
    objf = (util+div)
    return objf

def objf_func_new_set_d(S, X, Y):
    new_S = S.copy()
    new_S.remove(X)
    new_S.append(Y)
    util = util_func_d(new_S)
    div = div_func_d(new_S)
    objf = (util+div)
    return objf

def create_S_d(S, X, Y):
    new_S = S.copy()
    new_S.remove(X)
    new_S.append(Y)
    return new_S

In [8]:
def Greedy_Return_df(df, k):
    df_greedy = df.drop(['Utility'],axis=1)
    dataset = df_greedy.reset_index(drop=True)
    data1 = dataset.values.tolist()
    X = data1.copy()
    S = most_distant_two_views(data1)
    X.remove(S[0])
    X.remove(S[1])
    i = len(S)


    while i < k:    
        item = dist(X, S)
        #print(item)
        if item not in S:
            S.append(item)
            X.remove(item)
        else:
            pass

        i = i + 1  

    df_gh = pd.DataFrame(S, columns=['Attributes', 'Meassure', 'Function'])
    df_maxdiv = df_gh.merge(df, how='left')

    return df_maxdiv

In [9]:
k = 5

In [10]:
# df_greedy = Greedy_Return_df(df,k)
# df_greedy = df_greedy.drop(['Utility'],axis=1)

# Xdf = df.reset_index(drop=True)
# Xdf = Xdf.drop(['Utility'],axis=1)
# Retlist = df_greedy.values.tolist()

def div_new_set_d(S, X, Y):
    new_S = S.copy()
    new_S.remove(X)
    new_S.append(Y)
    #util = util_func_d(new_S)
    div = div_func_d(new_S)
    #objf = (util+div)
    return div

In [11]:
df_greedy = Greedy_Return_df(df,k)
Xdf = df.reset_index(drop=True)
Retlist = df_greedy.values.tolist()

X = Xdf.values.tolist()



#get the original X = X -S
Retlist_X = Retlist.copy()

for i in range(0,len(Retlist_X)):
    X.remove(Retlist_X[i])

S = Retlist.copy()

for i in range(0,len(X)):
    S_ = S.copy()
    for j in range(0,len(S)):
        if objf_func_d(S_) < objf_func_new_set_d(S, S[j], X[i]):
            new_S = create_S_d(S, S[j], X[i])
            S_ = new_S.copy()
            #print(S_)
    if objf_func_d(S_) > objf_func_d(S):
        S = S_.copy()
    #print("X ke {}".format(i))
objf = objf_func_d(S)

In [32]:
S

[['carrier', 'departurdelay', 'SUM', 0.3295100743406118],
 ['origin', 'arrivaldelay', 'STD', 0.33954727854007566],
 ['destination', 'arrivaldelay', 'MIN', 0.3616480582740928],
 ['destination', 'departurdelay', 'AVG', 0.46063951908037293],
 ['destination', 'weatherdelay', 'SUM', 0.42636531014458967]]

In [58]:
def diversity(a,b):
    la = list(a)
    lb = list(b)
    v1 = get_value_d(la[0],lb[0])
    v2 = get_value_d(la[1],lb[1])
    v3 = get_value_d(la[2],lb[2])
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    +
    return res

In [59]:
a = ['carrier', 'departurdelay', 'SUM']

In [60]:
b = ['destination', 'departurdelay', 'MIN']

In [61]:
diversity(a,b)

0.6666666666666666

In [24]:
import math
max_bound = math.sqrt(2)

In [25]:
max_bound

1.4142135623730951

In [31]:
first_list = [1, 2, 2, 5]
second_list = [2, 5, 7, 9]

in_first = set(first_list)
in_second = set(second_list)

in_first | in_second

{1, 2, 5, 7, 9}

In [30]:
in_second_but_not_in_first

{7, 9}

In [ ]:
alpha = 0.4

maxDI =(alpha*max_bound) + ((1-alpha)*mean)
new = 0.2 

maxDI =(alpha*new) + ((1-alpha)*maxDI)

In [ ]:
maxDI

In [ ]:
import numpy as np
from scipy.stats import truncnorm
import matplotlib.pyplot as plt

scale = 3.
range = 10
size = 1000

X = truncnorm(a=1, b=1000, scale=scale).rvs(size=size)

In [ ]:
X = X.round().astype(int)

In [ ]:
from numpy import random
N = random.choice(X,300)

In [ ]:
N

In [ ]:
N.mean()

In [ ]:
N.std()

In [ ]:
N.mean() + (3*N.std())

In [ ]:
X.max()

In [ ]:
def diversity(a,b):
    la = list(a)
    lb = list(b)
    v1 = get_value_d(la[0],lb[0])
    v2 = get_value_d(la[1],lb[1])
    v3 = get_value_d(la[2],lb[2])
    res = float((((v1*3)+(v2*2)+(v3*1))/3)/2)
    return res

In [ ]:
a = ['carrier', 'departurdelay', 'AVG']

In [ ]:
b =  ['carrier', 'departurdelay', 'MAX']

In [ ]:
diversity(a,b)